In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.io import fits
import myutils
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [69]:
coords_df = pd.read_csv("./data/processed/cut_crossmatched_objects.csv", index_col=0)[['ra_des','dec_des']]
BAND_ORDER = {"g": 0, "r": 1, "i": 2, "z": 3, "Y": 4}
all_imgs_array = np.zeros((len(coords_df), 28, 28, 5))
failed_object_mask = np.zeros(len(coords_df)).astype(bool)
# NOTE: remove [:2] from iteration
for i, (coadd_id, (ra, dec)) in tqdm(enumerate(coords_df[:2].iterrows()), total=len(coords_df)):
    FAILED_OBJECT_FLAG = False
    url_list = myutils.fetch_object_urls(ra, dec)
    if len(url_list) != 5:
        FAILED_OBJECT_FLAG = True
        failed_object_mask[i]=True
        break
    obj_array = np.zeros((28,28,5), dtype=np.float32)
    for url in url_list:
        if FAILED_OBJECT_FLAG:
            break
        raw_img = fits.open(url, cache=False)[0].data
        # TODO: What to do if image opening fails? In what ways does it fail?
        processed_img = myutils.crop_image(raw_img, px=28)
        if processed_img is None:
            FAILED_OBJECT_FLAG = True
            failed_object_mask[i]=True
            break
        band = myutils.band_from_url(url)
        obj_array[:, :, BAND_ORDER[band]] = processed_img
    if not FAILED_OBJECT_FLAG:
        all_imgs_array[i,:,:,:] = obj_array

success_coadd_ids = coords_df.index[~failed_object_mask]
success_imgs_array = all_imgs_array[~failed_object_mask]

  0%|          | 2/6566 [00:41<37:57:46, 20.82s/it]


In [67]:
failed_object_mask = np.random.randint(0,200,(len(coords_df)))
failed_object_mask = np.array([0 if flag else 1 for flag in failed_object_mask]).astype(bool)
all_imgs_array = np.abs(np.random.normal(size=(len(coords_df), 28, 28, 5))) * 1e4
success_imgs_array = all_imgs_array[~failed_object_mask]
success_coadd_ids = coords_df.index[~failed_object_mask]

In [12]:
import os
import numpy as np
import myutils

BAND_ORDER = {"g": 0, "r": 1, "i": 2, "z": 3, "Y": 4}

with open("./data/external/img_url_list.txt", "r") as f:
    url_filetext = f.read()

urls_list_by_object = url_filetext.split("#")[1:]
num_objs = len(urls_list_by_object)
coadd_ids = np.zeros(num_objs)
failed_mask = np.zeros(num_objs).astype(bool)
imgs = np.zeros((num_objs, 28, 28, 5))

DOWNLOADED_FILENAME = "./data/external/wget_file"

In [15]:
for i, object_url_string in enumerate(urls_list_by_object[:5]):
    FAILED_OBJECT_FLAG = False  # Innocent until proven guilty

    lines = object_url_string.split()  # ID & list of URLs for a particular object
    coadd_ids[i] = lines[0]  # Extra char at the start?
    urls = lines[1:]

    for url in urls:
        band = myutils.band_from_url(url)  # g/r/i/z/Y

        os.system(
            f"wget -nv -O {DOWNLOADED_FILENAME} '{url}'"
        )  # Downloads file TODO: Could go wrong?
        raw_img = myutils.fetch_image(
            DOWNLOADED_FILENAME
        )  # Extracts image from fits file TODO: Could go wrong?
        processed_img = myutils.crop_image(raw_img)  # Crops image to 28x28
        if processed_img is None:
            FAILED_OBJECT_FLAG = True
            failed_mask[i] = True
            break

        imgs[i, :, :, BAND_ORDER[band]] = myutils.crop_image(
            myutils.fetch_image(DOWNLOADED_FILENAME)
        )

FileNotFoundError: [Errno 2] No such file or directory: './data/external/wget_file'

In [14]:
url

'https://datalab.noirlab.edu/svc/cutout?col=des_dr2&siaRef=DES2138-4249_r4575p01_Y.fits.fz&extn=1&POS=324.85072599999995,-42.589015&SIZE=0.0028297437687020643,0.0020833333333333333'